In [2]:
!pip install -q euriai[langgraph]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.5 MB/s eta 0:00:00


In [3]:
from euriai.langgraph import EuriaiLangGraph

In [4]:
EURI_API_KEY = "euri-....."
FILE_NAME= "rag_contenxt.txt"

In [5]:
graph = EuriaiLangGraph(
    api_key= EURI_API_KEY,
    default_model= "gpt-4.1-nano"
)

In [6]:
from pathlib import Path
def reseracher_node_fun(state):
    query  = state["input"]
    file_path = FILE_NAME
    context = Path(file_path).read_text()

    return {
        "query":query,
        "context": context[:3000]
    }

In [7]:
graph.add_function_node("extractoin_and_query",reseracher_node_fun)

Added function node: extractoin_and_query


In [8]:
graph.get_graph_structure()

{'nodes': ['extractoin_and_query'],
 'ai_nodes': [],
 'edges': [],
 'conditional_edges': [],
 'entry_point': None,
 'finish_point': None}

In [9]:
graph.add_ai_node(
    "synthesizer",
    prompt_template="summarize the following document input that i hvae give to you in veyr very detailed:\n {context}"
)

Added AI node: synthesizer (model: gpt-4.1-nano)


In [10]:
graph.get_graph_structure()

{'nodes': ['extractoin_and_query', 'synthesizer'],
 'ai_nodes': ['synthesizer'],
 'edges': [],
 'conditional_edges': [],
 'entry_point': None,
 'finish_point': None}

In [11]:
def classify_topic(state):
    text = state["context"].lower()
    if "llm" in text:
        topic = "LLM Workflows"
    elif "rag" in text:
        topic = "RAG Pipelines"
    elif "langchain" in text:
        topic = "LangChain Integration"
    else:
        topic = "General AI"

    return {**state, "topic": topic}

In [12]:
graph.add_function_node('classifier',classify_topic)

Added function node: classifier


In [13]:
graph.get_graph_structure()

{'nodes': ['extractoin_and_query', 'synthesizer', 'classifier'],
 'ai_nodes': ['synthesizer'],
 'edges': [],
 'conditional_edges': [],
 'entry_point': None,
 'finish_point': None}

In [14]:
graph.add_ai_node(
    "finalizer",
    prompt_template={
        "User Query: {query}\n\nTopic: {topic}\n\nContext Summary: {context}\n\n"
        "Answer the query clearly using all context above:"
    }
)

Added AI node: finalizer (model: gpt-4.1-nano)


In [15]:
graph.get_graph_structure()

{'nodes': ['extractoin_and_query', 'synthesizer', 'classifier', 'finalizer'],
 'ai_nodes': ['synthesizer', 'finalizer'],
 'edges': [],
 'conditional_edges': [],
 'entry_point': None,
 'finish_point': None}

In [16]:
graph.set_entry_point('extractoin_and_query')
graph.add_edge('extractoin_and_query',"synthesizer")

Set entry point: extractoin_and_query
Added edge: extractoin_and_query -> synthesizer


In [17]:
graph.add_edge("synthesizer",'classifier')

Added edge: synthesizer -> classifier


In [18]:
graph.add_edge('classifier',"finalizer")

Added edge: classifier -> finalizer


In [19]:
graph.set_finish_point("finalizer")

Set finish point: finalizer


In [20]:
graph.get_graph_structure()

{'nodes': ['extractoin_and_query', 'synthesizer', 'classifier', 'finalizer'],
 'ai_nodes': ['synthesizer', 'finalizer'],
 'edges': [('extractoin_and_query', 'synthesizer'),
  ('synthesizer', 'classifier'),
  ('classifier', 'finalizer')],
 'conditional_edges': [],
 'entry_point': 'extractoin_and_query',
 'finish_point': 'finalizer'}

In [21]:
result = graph.run({"input":"how can langgraph and euri be used to build a multi agent rag systme"})
print("final output:\n" ,result)

Graph compiled successfully
[values] {'input': 'how can langgraph and euri be used to build a multi agent rag systme'}
[updates] {'extractoin_and_query': {'query': 'how can langgraph and euri be used to build a multi agent rag systme', 'context': 'LangGraph is an open-source Python framework for building stateful, composable, and observable LLM workflows. It provides primitives for defining nodes, memory, edges, and agents. LangGraph supports integration with LangChain, LlamaIndex, and various vector stores and retrievers.\n\nYou can build multi-agent pipelines with memory, retrievers, tools, and streaming outputs. EURI AI integrates by powering each node with real-time inference using models like gpt-4.1-nano.\n'}}
[values] {'query': 'how can langgraph and euri be used to build a multi agent rag systme', 'context': 'LangGraph is an open-source Python framework for building stateful, composable, and observable LLM workflows. It provides primitives for defining nodes, memory, edges, and

ERROR:root:Error in AI node finalizer after 1 attempts: 'set' object has no attribute 'format'


[updates] {'synthesizer': {'query': 'how can langgraph and euri be used to build a multi agent rag systme', 'context': 'LangGraph is an open-source Python framework for building stateful, composable, and observable LLM workflows. It provides primitives for defining nodes, memory, edges, and agents. LangGraph supports integration with LangChain, LlamaIndex, and various vector stores and retrievers.\n\nYou can build multi-agent pipelines with memory, retrievers, tools, and streaming outputs. EURI AI integrates by powering each node with real-time inference using models like gpt-4.1-nano.\n', 'synthesizer_output': "Certainly! Here's a very detailed summary of the provided document:\n\nLangGraph Overview:\n- LangGraph is an open-source framework developed in Python designed to facilitate the construction of complex workflows involving large language models (LLMs). \n- Its architecture emphasizes statefulness, composability, and observability, enabling users to create modular and transparen

In [22]:
from IPython.display import display, Markdown, Latex
display(Markdown(result['query']))
display(Markdown(result['context']))

how can langgraph and euri be used to build a multi agent rag systme

LangGraph is an open-source Python framework for building stateful, composable, and observable LLM workflows. It provides primitives for defining nodes, memory, edges, and agents. LangGraph supports integration with LangChain, LlamaIndex, and various vector stores and retrievers.

You can build multi-agent pipelines with memory, retrievers, tools, and streaming outputs. EURI AI integrates by powering each node with real-time inference using models like gpt-4.1-nano.
